In [ ]:
###Imports
import pandas as pd
import os
import librosa
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
###Data Preprocessing
audio_dataset_path='/content/drive/MyDrive/UNI/HCI/TestClassifier/Test3/Geschnittene-wav'
metadata=pd.read_csv('/content/drive/MyDrive/UNI/HCI/TestClassifier/Test3/metadata UniStudie.csv')

#Extraction function
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

#Extracting features
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])


#Converting features into dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])

x=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

#Label encoder
labelencoder = LabelEncoder()
y= to_categorical(labelencoder.fit_transform(y))

#Spliting data for training and testing

x_train,x_test,y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
###Training model
model = KNeighborsClassifier(n_neighbors=1)
model.fit(x_train,y_train)
print(model.score(x_test, y_test))

In [ ]:
###Classification
filename="/content/drive/MyDrive/UNI/HCI/TestClassifier/Test3/Geschnittene-wav/T7/2.wav"
filename2="/content/drive/MyDrive/UNI/HCI/TestClassifier/Test3/Geschnittene-wav/T2/3.wav"
print((model.predict(features_extractor(filename).reshape(1,-1))))
print((model.predict(features_extractor(filename2).reshape(1,-1))))